In [29]:
import numpy as np;
import tensorflow as tf
from numpy import *

In [30]:
#question
glove_matrix  = np.load("glove.npy")
vocab = np.fromfile("vocab.dat",dtype=float)
question_file = open("question_id.txt");
question_string = question_file.read();
questions = question_string.split("\n")
q_train=np.zeros((size(questions),100))
for i in range(100):
    st = questions[i].split(" ");
    st = list(map(int, st)) 
    q_train[i] =np.sum(glove_matrix[[st]],axis = 0)

In [31]:
#comprehension
context_file = open("context_id.txt");
context_string = context_file.read();
contexts = context_string.split("\n")
#size(contexts)
stc= [];
c_train=np.zeros((size(contexts),100))
for i in range(100):
    st1 = contexts[i].split(" ");
    stc.append(list(map(int, st1)));
    c_train[i] =np.sum(glove_matrix[[stc[i]]],axis = 0)

In [32]:
#answer
answer_file = open("span.txt");
answer_string = answer_file.read();
answers = answer_string.split("\n")
a_train=np.zeros((size(answers),100))
for i in range(100):
    ans = answers[i].split(" ");
    ans1 = list(map(int, ans)) 
    st_answer = stc[i][ans1[0]:ans1[1]+1]
    a_train[i] =np.sum(glove_matrix[st_answer],axis = 0)

In [33]:
train = np.concatenate((c_train, q_train), axis=1)
c_train.shape
q_train.shape
train.shape 

(81404, 200)

In [34]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [35]:
n_hidden_1 = 20
n_input = train.shape[1]
n_classes = a_train.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

In [41]:
training_epochs = 20
display_step = 10
batch_size = 10

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [42]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))

In [43]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [47]:
x_train1 = train
y_train1 = a_train
train_size = 0.9
train_cnt = int(floor(x_train1.shape[0] * train_size))
print(train_cnt)
x_train = x_train1[0:train_cnt]
y_train = y_train1[0:train_cnt]
x_test = x_train1[train_cnt:]
y_test = y_train1[train_cnt:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

73263
(73263, 200) (73263, 100) (8141, 200) (8141, 100)


In [48]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
        print(epoch)
    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(x_test.shape,accuracy,y_test.shape)
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Epoch: 0001 cost= -78.128249343
0
1
2
3
4
5
6
7
8
9
Epoch: 0011 cost= -178.762031207
10
11
12
13
14
15
16
17
18
19
Optimization Finished!
(8141, 200) Tensor("Mean_6:0", shape=(), dtype=float32) (8141, 100)
Accuracy: 0.0


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`